In [15]:
# 📚 Importations
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pickle

# 🔧 Paramètres de connexion
host = "localhost"
port = "5432"
user = "postgres"
password = "maher123"

DIMENSIONS_DB = "DIMENSIONS"  # Base contenant DimDate et DimShops
FACTS_DB = "FACTS"            # Base contenant Fact_Sales

# 🔄 Fonction pour charger les données
def load_data():
    dim_engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{DIMENSIONS_DB}")
    fact_engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{FACTS_DB}")
    
    with dim_engine.connect() as dim_conn, fact_engine.connect() as fact_conn:
        print("✅ Connected to DIMENSIONS and FACTS.")
        
        dim_date = pd.read_sql('SELECT "date_id", "full_date", "year", "month", "day" FROM public."DimDate";', dim_conn)
        dim_shops = pd.read_sql('SELECT "Pk_Shops", "shopname" FROM public."DimShops";', dim_conn)
        fact_sales = pd.read_sql('SELECT "Fk_Shops", "Fk_Date", "TotalAmount" FROM public."Fact_Sales";', fact_conn)
        
    return dim_date, dim_shops, fact_sales

# 📥 Chargement des données
dim_date, dim_shops, fact_sales = load_data()

# 🔗 Forcer les types avant la fusion
fact_sales["Fk_Shops"] = fact_sales["Fk_Shops"].astype(str)
dim_shops["Pk_Shops"] = dim_shops["Pk_Shops"].astype(str)

fact_sales["Fk_Date"] = fact_sales["Fk_Date"].astype(str)
dim_date["date_id"] = dim_date["date_id"].astype(str)

# 🧹 Vérification des valeurs manquantes
print("✅ Vérification des valeurs manquantes :")
print(fact_sales.isnull().sum())
print(dim_shops.isnull().sum())
print(dim_date.isnull().sum())

# 🔗 Fusion des données
df = fact_sales \
    .merge(dim_shops, left_on="Fk_Shops", right_on="Pk_Shops", how="inner") \
    .merge(dim_date, left_on="Fk_Date", right_on="date_id", how="inner")

# 📊 Vérification de la taille du DataFrame après fusion
print(f"✅ Shape of merged dataframe: {df.shape}")

# 🏗️ Feature Engineering
df['shopname'] = df['shopname'].astype('category')
df['shopname_encoded'] = df['shopname'].cat.codes

df['full_date'] = pd.to_datetime(df['full_date'], errors='coerce')
df.dropna(subset=['full_date'], inplace=True)

df['year'] = df['full_date'].dt.year
df['month'] = df['full_date'].dt.month
df['day'] = df['full_date'].dt.day

# 🧠 Préparation du dataset
X = df[['shopname_encoded', 'year', 'month', 'day']]
y = df['TotalAmount']

# 🧪 Split et entraînement
if X.shape[0] > 0:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # 🎯 Modèles
    lr_model = LinearRegression()
    lr_model.fit(X_train, y_train)

    rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
    rf_model.fit(X_train, y_train)

    # 📈 Évaluation
    print(f"Linear Regression RMSE: {mean_squared_error(y_test, lr_model.predict(X_test), squared=False):.2f}")
    print(f"Random Forest RMSE: {mean_squared_error(y_test, rf_model.predict(X_test), squared=False):.2f}")

    # 💾 Sauvegarde des modèles
    with open('linear_model.pkl', 'wb') as f:
        pickle.dump(lr_model, f)

    with open('rf_model.pkl', 'wb') as f:
        pickle.dump(rf_model, f)

    print("✅ Modèles enregistrés avec succès.")
else:
    print("❌ Aucune donnée disponible après la fusion, le modèle ne peut pas être entraîné.")


✅ Connected to DIMENSIONS and FACTS.
✅ Vérification des valeurs manquantes :
Fk_Shops       0
Fk_Date        0
TotalAmount    0
dtype: int64
Pk_Shops    0
shopname    0
dtype: int64
date_id      0
full_date    0
year         0
month        0
day          0
dtype: int64
✅ Shape of merged dataframe: (278800, 10)
Linear Regression RMSE: 3178.47
Random Forest RMSE: 998.40
✅ Modèles enregistrés avec succès.
